In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

In [ ]:
os.environ["OMP_NUM_THREADS"] = "10" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "10" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "10" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "10" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "10" # export NUMEXPR_NUM_THREADS=1

In [ ]:
from utils.dataset import load_mnist784, load_mnist64, get_gaussian
import matplotlib.pyplot as plt
import numpy as np
from utils.tm import get_2d_tm

In [ ]:
digits = load_mnist784(4)

In [ ]:
img = digits[0].reshape(28,28)

plt.imshow(img, cmap="Blues")

In [ ]:
digits = load_mnist64(4)

In [ ]:
img = digits[0].reshape(8,8)

plt.imshow(img, cmap="Blues")

In [ ]:
d = len(digits[0])
d

In [ ]:
d = 100
digits, bar_true = get_gaussian(d, 100)

In [ ]:
C = get_2d_tm(int(np.sqrt(d)))
C.shape # d^2

In [ ]:
from oracles.point import OTProblemOracle

In [ ]:
T = 100 # number of all 
assert T <= len(digits)
n = 50
assert n <= T

In [ ]:
np.random.seed(30)

In [ ]:
random_digits = digits[np.random.choice(len(digits), size=T, replace=False), :]

In [ ]:
oracles = [OTProblemOracle(random_digits[i], C) for i in range(T)]

In [ ]:
from oracles.oracle import OperatorOracle

In [ ]:
F1 = OperatorOracle(oracles, d, n)

In [ ]:
F = OperatorOracle(oracles, d, T)

In [ ]:
from methods.paus import PAUS

In [ ]:
method = PAUS(F, F1, 1, 1000, True)

In [ ]:
delta = 2 * (T-n)/T

In [ ]:
z_star, history = method.fit(delta, bar_true=bar_true)

In [ ]:
import pickle

In [ ]:
with open(f"paus_gaussian_history.pkl", "wb") as f:
    pickle.dump(history, f)
with open(f"paus_gaussian_z.pkl", "wb") as f:
    pickle.dump(z_star, f)

In [ ]:
digits = load_mnist784(4)

In [ ]:
d = len(digits[0])
d

In [ ]:
C = get_2d_tm(int(np.sqrt(d)))
C.shape # d^2

In [ ]:
T = 100 # number of all 
assert T <= len(digits)
n = 50
assert n <= T

In [ ]:
np.random.seed(30)

In [ ]:
random_digits = digits[np.random.choice(len(digits), size=T, replace=False), :]

In [ ]:
oracles = [OTProblemOracle(random_digits[i], C) for i in range(T)]

In [ ]:
F1 = OperatorOracle(oracles, d, n)

In [ ]:
F = OperatorOracle(oracles, d, T)

In [ ]:
method = PAUS(F, F1, 1, 1000, True)

In [ ]:
delta = 2 * (T-n)/T

In [ ]:
z_star, history = method.fit(delta, bar_true=bar_true)

In [ ]:
with open(f"paus_mnist_history.pkl", "wb") as f:
    pickle.dump(history, f)
with open(f"paus_mnist_z.pkl", "wb") as f:
    pickle.dump(z_star, f)